In [1]:
#import libraries
import pandas as pd
import tensorflow as tf
from keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from sklearn import metrics
import time


In [2]:
wins = [x for x in range(10,101,10)]
tab = []

for t in wins:
    x = np.load('../data/training_data/ddaccgz_' + str(t) + '_input_x.npy')
    y = np.load('../data/training_data/ddaccgz_' + str(t) + '_input_y.npy')

    result = np.where(y==1) # Identify take over instances
    all_indices = list(range(y.shape[0]))
    ntc_index = [i for i in all_indices if i not in result[0]] # Identify non-take over instances
    tc_index = list(result[0])

    # Take a subset of the non-take over instances such that class ratio is 1:1
    tc_idx_size = len(tc_index)
    random.shuffle(ntc_index)
    ntc_data_idx = ntc_index[:tc_idx_size]

    # Combine and shuffle the indices
    idx = ntc_data_idx+tc_index
    random.shuffle(idx)

    X = x[idx]
    Y = y[idx]
    Y = Y.reshape(-1,1)
    
    x_t, x_test, y_t, y_test = train_test_split(X, Y, test_size=0.25, shuffle=True)
    x_train, x_val, y_train, y_val = train_test_split(x_t, y_t, test_size=0.25, shuffle=True)

    tf.random.set_seed(0)

    model = models.Sequential()
    model.add(layers.Conv2D(8, (3,3), activation='relu', input_shape = (X.shape[1],X.shape[2],1)))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(16, (3,3), activation='relu'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(16, activation='tanh'))
    model.add(layers.Dense(8, activation='tanh'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    start = time.time()

    history = model.fit(x_train, y_train, epochs=20, validation_data=(x_val, y_val), verbose=0)

    end = time.time()

    plt.plot(history.history['accuracy'], label='train accuracy')
    plt.plot(history.history['val_accuracy'], label='validation accuracy')
    plt.plot(history.history['loss'], label='train loss')
    plt.plot(history.history['val_loss'], label='validation loss')
    plt.xlabel('# of epochs')
    plt.ylabel('accuracy / loss')
    plt.ylim([0,1])
    plt.legend(loc='center right')
    plt.title(t)
    plt.savefig('../data/results/figures/ddaccgz/' + str(t) + '.png', dpi=800)
    plt.close()

    y_pred = model.predict(x_test, verbose=0)
    for i in y_pred:
        if i[0] >= 0.5:
            i[0] = 1
        else:
            i[0] = 0

    cf = metrics.confusion_matrix(y_test, y_pred)
    total=sum(sum(cf))
    accuracy = (cf[0,0]+cf[1,1])/total
    specificity = cf[0,0]/(cf[0,0]+cf[0,1])
    sensitivity = cf[1,1]/(cf[1,0]+cf[1,1]) #recall
    precision = cf[1,1]/(cf[0,1]+cf[1,1])
    f1 = 2*(1/((1/precision)+(1/sensitivity)))
    ti = end - start

    tp = [t, accuracy, specificity, sensitivity, precision, f1, ti]
    tab.append(tp)

table = pd.DataFrame(tab, columns=['datatype', 'accuracy', 'specificity', 'sensitivity', 'precision', 'f1-score', 'train_time'])
table.to_excel('../data/results/accuracy_tables/ddaccgz.xlsx', index=False)


2023-02-26 16:30:05.291737: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-26 16:30:05.291846: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-02-26 16:30:05.442509: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-26 16:30:05.613407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 16:30:06.690587: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 16:30:24.098486: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 16:30:26.439669: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 16:30:28.175893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 16:30:59.574664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-26 16:31:02.774749: I tensorflow/core/grappler/optimizers/cust